# Create a monthly binned posts counts of all StackOverflow posts

- Create a Parquet file that contains only id/creation date series
- This allows us to plot a baseline of StackOverflow raise and decline 
  over time
- We can use this to compare blockchain question popularity to overall StackOverflow popularity
- Run time ~15 minutes

In [1]:
import pandas as pd
from tqdm.auto import tqdm
from pandas.io.parsers.readers import TextFileReader

chunk_size = 2**16  # 64k rows at a time
result_df: pd.DataFrame = None
matched_chunks: list[pd.DataFrame] = []
match_count = row_count = 0

with tqdm() as progress_bar:

    reader: TextFileReader

    rows_read = 0

    with pd.read_csv("csv/Posts.csv", chunksize=chunk_size) as reader:
        chunk: pd.DataFrame
        for chunk in reader:
                        
            # Find posts in this chunk that match our tag filter
            matched_chunk = pd.DataFrame()
            # Parse the dates 
            # https://stackoverflow.com/a/61959823/315168
            matched_chunk["CreationDate"] = pd.to_datetime(chunk["CreationDate"], format='ISO8601')
            matched_chunk["Id"] = chunk["Id"]
            matched_chunk.set_index("Id")
            
            matched_chunks.append(matched_chunk)
            row_count += len(chunk)

            last = chunk.iloc[-1]

            # Show the date where the filter progres is going.
            # We are finished when reaching 2023-06
            progress_bar.set_postfix({
                "Date": last["CreationDate"],      
                "Total rows": f"{row_count:,}",
            })

            # Display rows read as a progress bar,
            # but we do not know the end
            progress_bar.update(len(chunk))


result_df = pd.concat(matched_chunks)

0it [00:00, ?it/s]

In [2]:
# Write raw output 
result_df.to_parquet("all-creation-dates.parquet")

In [24]:
import itertools
# Count posts by month for a smaller dataset
# https://stackoverflow.com/a/55726226/315168

# dt_indexed_df = result_df.set_index("CreationDate")

dt_indexed_df = result_df

# https://stackoverflow.com/a/56280791/315168
grouped_df = result_df.groupby([pd.Grouper(key='CreationDate', freq='MS')])

# Create a dataframe that contains group -> count mappings
post_counts_month_df = pd.DataFrame()
post_counts_month_df["posts_by_period"] = grouped_df.count()


In [26]:

# Do manual inspection of data
for month_start, month_posts in itertools.islice(grouped_df, 5):
    print(f"Month {month_start}, total {len(month_posts)}")

# Do inspection of processed data frame
for idx, row in itertools.islice(post_counts_month_df.iterrows(), 5):
    print(f"{idx}: {row['posts_by_period']}")

post_counts_month_df.to_parquet("post_counts_month.parquet")

Month (Timestamp('2008-07-01 00:00:00'),), total 6
Month (Timestamp('2008-08-01 00:00:00'),), total 18508
Month (Timestamp('2008-09-01 00:00:00'),), total 74271
Month (Timestamp('2008-10-01 00:00:00'),), total 68187
Month (Timestamp('2008-11-01 00:00:00'),), total 53338
2008-07-01 00:00:00: 6
2008-08-01 00:00:00: 18508
2008-09-01 00:00:00: 74271
2008-10-01 00:00:00: 68187
2008-11-01 00:00:00: 53338


In [ ]:
# Now quaterly
grouped_df = result_df.groupby([pd.Grouper(key='CreationDate', freq=pd.offsets.QuarterBegin())])
post_counts_quarterly_df = pd.DataFrame()
post_counts_quarterly_df["posts_by_period"] = grouped_df.count()
post_counts_quarterly_df.to_parquet("post_counts_quarterly.parquet")